In [1]:
# import dependencies
import pandas as pd
import nltk

# will need to uncomment the following the first time you run
# nltk.download('punkt')

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# import movies
movies = pd.read_csv("../ml-latest-small/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# import ratings
ratings = pd.read_csv("../ml-latest-small/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# import tags
# do we want to use this?
tags = pd.read_csv("../ml-latest-small/tags.csv")
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [5]:
# Make each user id a unique row?

# merge movies and ratings
df = pd.merge(ratings, movies, on="movieId")
df.sort_values("userId").head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
12117,1,2389,2.0,964983094,Psycho (1998),Crime|Horror|Thriller
12126,1,2395,5.0,964981093,Rushmore (1998),Comedy|Drama
12182,1,2406,4.0,964982310,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
776,1,70,3.0,964982400,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller


In [6]:
df['year'] = df['title'].str.extract('.*\((.*)\).*')
df.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [7]:
# prep the year column to convert to int
df = df.replace(to_replace='2006–2007', value='2007')
df = df.fillna(0)
df.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [8]:
# split the genres out from the pipe delimeters and extract year from title into new column
df['genres'] = df['genres'].str.lower()
df['genres'] = df['genres'].str.split("|")

In [9]:
df['genres']=[" ".join(word) for word in df['genres'].values]

In [22]:
v = TfidfVectorizer()
x = v.fit_transform(df['genres'])
df['genresVect'] = list(x.toarray())
df.head()

,userId,movieId,rating,timestamp,title,genres,year,genresVect
0,1,1,4,964982703,Toy Story (1995),adventure animation children comedy fantasy,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
1,5,1,4,847434962,Toy Story (1995),adventure animation children comedy fantasy,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
2,7,1,4,1106635946,Toy Story (1995),adventure animation children comedy fantasy,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
3,15,1,2,1510577970,Toy Story (1995),adventure animation children comedy fantasy,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
4,17,1,4,1305696483,Toy Story (1995),adventure animation children comedy fantasy,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."


In [23]:
# convert the tags or the genres to numerical using hashing?

In [24]:
# convert to int
df['rating'] = df['rating'].astype(int) 
target = df["rating"]

In [25]:
df['userId'] = df['userId'].astype(int) 
df['movieId'] = df['movieId'].astype(int) 
df['year'] = df['year'].astype(int) 

In [26]:
data = df[["userId", "movieId", "year", "genresVect"]]
feature_names = data.columns
data.head()

,userId,movieId,year,genresVect
0,1,1,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
1,5,1,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
2,7,1,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
3,15,1,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."
4,17,1,1995,"[0.0, 0.3638850174997462, 0.5497352121009116, ..."


In [27]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [28]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

ValueError: setting an array element with a sequence.

In [ ]:
importances = rf.feature_importances_
importances

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

In [ ]:
sns.barplot(x=importances, y=feature_names)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Features by Importance")
plt.legend()
plt.show()